In [20]:
import pandas as pd
from utils import * 
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import random
import emoji

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs()

maxLen = 50

X, Y = read_csv(word_to_index, 10, maxLen)
# print(X.shape)
# Z = list(zip(X,Y))
# random.shuffle(Z)

# X, Y = zip(*Z)
# X, Y = list(X), list(Y)
for i in range(len(Y)):
    Y[i] -= 1
X_length = len(X)
max_mult = X_length//256
print(X_length)

6741


In [4]:
X.shape
# for sentence in X:
#     sentence_to_words = sentence.lower().split(".")
#     sentence_to_words = sentence_to_words[0].split()
#     list_len.append(len(sentence_to_words))
# print(list_len[:10])
# print(max(list_len))
# print(sum(list_len) / len(list_len))
# list_triee = sorted(list_len)
# for i in range(10):
#     print(list_triee[i * len(list_triee)//10])

(6741,)

In [5]:

X_train, Y_train = X[0:256*(max_mult - 2)], Y[0:256*(max_mult - 2)]
X_test, Y_test = X[256*(max_mult - 2):256*(max_mult - 1)], Y[256*(max_mult - 2):256*(max_mult - 1)]
X_valid, Y_valid = X[256*(max_mult - 1):256*max_mult], Y[256*(max_mult - 1):256*max_mult]

Y_train_oh = convert_to_oh(Y_train, C = 5)
Y_test_oh = convert_to_oh(Y_test, C = 5)
Y_valid_oh = convert_to_oh(Y_valid, C = 5)

# X_train, Y_train = read_csv2('data/train_emoji.csv')
# X_test, Y_test = read_csv2('data/tesss.csv')
# X_valid, Y_valid = X_test, Y_test
# Y_train_oh = convert_to_oh(Y_train, C = 5)
# Y_test_oh = convert_to_oh(Y_test, C = 5)
# Y_valid_oh = convert_to_oh(Y_test, C = 5)

print( len(X_train))

6144


In [6]:
for i in range(20):
    print(X_test[i], Y_test[i])

beautiful family resort just returned week christmas holiday family 9 ranging age 12 64  4
excellent value money returned majestic week like review tripadvisor reviews useful deciding hotel to  4
mixed rating property quite tough outstanding poor rating various reasons , buildings pools spa beach outstanding  2
majestic colonial just returned trip stayed majestic july 30 june 8  3
not ready visitors place wide berth months year not ready offer 5 star service fact expecting large 5 star hotel want n ' t think place really offer 5 star experience lot growing pains lot learn customer satisfaction , wife spent dec  0
fabulous hotel great hotel , rooms amazing large great views clean , service excellent , location good close subway ,   4
perfect hotel just stayed week prince park tower tokyo hotel rate hotel excellent especially room rate paid euro , comparison year stayed ana hotel expensive did not offer quality  4
family run personal nice area , older hotel basic 1 night stay , result ha

In [7]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    """
    m = X.shape[0]

    X_indices = np.zeros((m, max_len))
    for i in range(m):

        sentence_to_words = X[i].lower().split(".")
        sentence_to_words = sentence_to_words[0].split()
        j = 0

        for w in sentence_to_words:
            X_indices[i, j] = word_to_index[w]
            j += 1
    return X_indices

In [8]:
X1 = np.array(["funny lol. test.", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

X1 = ['funny lol. test.' 'lets play baseball' 'food is ready for you']
X1_indices =
 [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [9]:

# m = X.shape[0]
# maxLen = 0
# for i in range(m):
#     sentence_to_words = X[i].lower().split(".")
#     sentence_to_words = sentence_to_words[0].split()
#     if len(sentence_to_words) > maxLen:
#         maxLen = len(sentence_to_words)
print("Sentences max length: ", maxLen)


Sentences max length:  50


In [10]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    """
    
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    

    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]


    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    # Build the embedding layer, it is required before setting the weights of the embedding layer. 
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [11]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


In [12]:
def commentary(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the commentary model's graph.
    
    """
    
    sentence_indices = Input(input_shape, dtype='int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = LSTM(128, return_sequences= True)(embeddings)
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences= False)(X)
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 5 units
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices, X)
    
    return model

In [13]:
model = commentary((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            20000050  
_________________________________________________________________
lstm (LSTM)                  (None, 50, 128)           91648     
_________________________________________________________________
dropout (Dropout)            (None, 50, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                

In [14]:
print("Model compile......")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model compile......


In [15]:
# for x in X_train:
#     print(len(x))
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)

In [16]:
print("Model fit......")
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 64, shuffle=True)
print("End of fitting......")

Model fit......
Epoch 1/50
96/96 [==============================] - 1s 9ms/step - loss: 1.3138 - accuracy: 0.4255
Epoch 2/50
96/96 [==============================] - 1s 8ms/step - loss: 1.1839 - accuracy: 0.4696
Epoch 3/50
96/96 [==============================] - 1s 8ms/step - loss: 1.1621 - accuracy: 0.4696
Epoch 4/50
96/96 [==============================] - 1s 8ms/step - loss: 1.1191 - accuracy: 0.4982
Epoch 5/50
96/96 [==============================] - 1s 8ms/step - loss: 1.0950 - accuracy: 0.4930
Epoch 6/50
96/96 [==============================] - 1s 8ms/step - loss: 1.0663 - accuracy: 0.5282
Epoch 7/50
96/96 [==============================] - 1s 8ms/step - loss: 1.0398 - accuracy: 0.5360
Epoch 8/50
96/96 [==============================] - 1s 8ms/step - loss: 1.0237 - accuracy: 0.5373
Epoch 9/50
96/96 [==============================] - 1s 9ms/step - loss: 0.9890 - accuracy: 0.5549
Epoch 10/50
96/96 [==============================] - 1s 9ms/step - loss: 0.9741 - accuracy: 0.5747
Epo

In [17]:
X_valid_indices = sentences_to_indices(X_valid, word_to_index, max_len = maxLen)
loss, acc = model.evaluate(X_valid_indices, Y_valid_oh)
print()
print("Test accuracy = ", acc)


8/8 [==============================] - 0s 3ms/step - loss: 2.3917 - accuracy: 0.4844

Test accuracy =  0.484375


In [62]:
C = 5
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
miss_match = 0
one_difference = 0
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(abs(num - Y_test[i]) >= 2):
        print(X_test[i])
        print(emoji.emojize(':orange_heart:')*(num+1), '=/=',emoji.emojize(':green_heart:')*(Y_test[i]+1))
        miss_match = miss_match +1
    elif abs(num - Y_test[i]) == 1:
        one_difference +=1
print('miss match = ' , miss_match,  "/" , len(X_test))
print('one star difference = ' , one_difference,  "/" , len(X_test))

family run personal nice area , older hotel basic 1 night stay , result having stop singapore way locations traveling stayed south east asia hotel 1 stays hotel bencoolen review property stay 
🧡🧡🧡🧡🧡 =/= 💚💚💚
home florence arrived long car trip angela awaiting , immediately showed room gave quick thorough instructions , able relax spacious cozy spotless room no delay , later directed neighborhood restaurant captured soul tuscany , did not use car stay minibus takes ponte vecchio block hotel gets minutes 
🧡 =/= 💚💚💚💚💚
gorgeous resort beautiful , grounds amazing , complaint not best single travelers inclusive n ' t area singles , small casino resort open night , bars , beach pools nice , staff moderately friendly helpful not overly ,  
🧡🧡 =/= 💚💚💚💚💚
just like dumb blonde , wife stayed ocean royal service suite april 21 28th 
🧡🧡🧡🧡🧡 =/= 💚💚
special celebration husband adult children just got week , stayed thanksgiving weekend celebrate 50th birthday , resort gorgeous , no roaches place clean , 

In [52]:

print('miss match = ' , miss_match,  "/" , len(X_test), '===>', (miss_match/len(X_test)*100), "%")
print('one star difference = ' , one_difference,  "/" , len(X_test), '===>', (one_difference/len(X_test)*100), "%")


miss match =  20 / 256 ===> 7.8125 %
one star difference =  93 / 256 ===> 36.328125 %
